In [2]:
import os
import traceback
import pandas as pd
import geopandas as gpd
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np

user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [4]:
sql ="""
select build_id, geom, prim_occ from structures.alabama limit 5000
"""
try:
    gdf = gpd.GeoDataFrame.from_postgis(sql, con=engine,geom_col='geom') # epsg 4326

    gdf = gdf.to_crs("ESRI:102003") # USA_Contiguous_Albers_Equal_Area_Conic Projection

    gdf["length"] = gdf.bounds["maxy"] - gdf.bounds["miny"]
    gdf["width"] = gdf.bounds["maxx"] - gdf.bounds["minx"]

    gdf["longest_side"] = gdf[["width", "length"]].max(axis=1)
    gdf["shortest_side"] = gdf[["width", "length"]].min(axis=1)

    gdf["ls_ratio"] = gdf["longest_side"] / gdf["shortest_side"]

    print(gdf.head())

except Exception:
    print(traceback.print_exc())

   build_id                                               geom  \
0   4334275  MULTIPOLYGON (((926418.058 -396760.560, 926413...   
1   4334278  MULTIPOLYGON (((897678.251 -399773.034, 897674...   
2   4334282  MULTIPOLYGON (((905553.145 -398966.557, 905544...   
3   4334284  MULTIPOLYGON (((909101.643 -398577.750, 909120...   
4   4334290  MULTIPOLYGON (((898209.862 -399706.191, 898216...   

                  prim_occ     length      width  longest_side  shortest_side  \
0   Single Family Dwelling  12.226814  10.896948     12.226814      10.896948   
1  Multi - Family Dwelling  20.161788  23.170877     23.170877      20.161788   
2   Single Family Dwelling  21.175417  17.696566     21.175417      17.696566   
3   Single Family Dwelling  23.943836  18.839774     23.943836      18.839774   
4   Single Family Dwelling  21.292324  14.180077     21.292324      14.180077   

   ls_ratio  
0  1.122040  
1  1.149247  
2  1.196583  
3  1.270920  
4  1.501566  
